In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Setup
options = webdriver.ChromeOptions()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

# Read symbols
symbols = []
with open("/Users/farazahmed/Documents/data2.csv", 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        symbols.append(row[0])

data = []

for idx, symbol in enumerate(symbols[:100], 1):  
    url = f"https://www.tradingview.com/symbols/{symbol}/"
    driver.get(url)
    time.sleep(3)

    try:
        # Example: Get company description or header
        title = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "tv-symbol-header__first-line")))
        description = driver.find_element(By.CLASS_NAME, "tv-symbol-profile__description").text

        data.append([symbol, title.text, description])
        print(f"✅ {idx}. Fetched data for {symbol}")
    except Exception as e:
        print(f"❌ {idx}. Failed for {symbol} — {e}")
        data.append([symbol, "N/A", "N/A"])

# Save to CSV
with open("/Users/farazahmed/Documents/task3.csv", 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Symbol', 'Title', 'Description'])
    writer.writerows(data)

print("\n✅ Done! Saved detailed data.")
driver.quit()

KeyboardInterrupt: 

In [9]:
import csv
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# ─── Setup Chrome WebDriver ─────────────────────────────────────────────────────
options = webdriver.ChromeOptions()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
# options.add_argument('--headless')  # uncomment if you don't need to see the browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

# ─── Read symbols from CSV ──────────────────────────────────────────────────────
symbols = []
with open("/Users/farazahmed/Documents/data2.csv", newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # skip header
    for row in reader:
        if row:
            symbols.append(row[0].strip())

print(f"Total symbols read: {len(symbols)}")

# ─── Helper to pull out fundamentals by label ───────────────────────────────────
def get_fundamental(label_text):
    """
    On TV overview pages, fundamentals appear in a widget
    as pairs of <div class="tv-widget-fundamentals__label">Label</div>
    and <div class="tv-widget-fundamentals__value">Value</div>.
    This helper finds the label and returns its adjacent value.
    """
    try:
        label = driver.find_element(
            By.XPATH,
            f"//div[contains(@class,'tv-widget-fundamentals__label') and normalize-space()='{label_text}']"
        )
        value = label.find_element(
            By.XPATH,
            "./following-sibling::div[contains(@class,'tv-widget-fundamentals__value')]"
        )
        return value.text.strip()
    except NoSuchElementException:
        return "N/A"

# ─── Main scrape loop ───────────────────────────────────────────────────────────
output = []
for idx, sym in enumerate(symbols, 1):
    if ':' in sym:
        exch, ticker = sym.split(':', 1)
    else:
        exch, ticker = 'NYSE', sym  # default changed to NYSE

    url = f"https://www.tradingview.com/symbols/{exch}-{ticker}/"
    driver.get(url)
    
    try:
        # Wait for the fundamentals widget to load
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".tv-widget-fundamentals")))
        # Optional short sleep to reduce bot detection risk
        time.sleep(random.uniform(1.0, 2.0))

        row = {
            "Symbol": sym,
            "Next report date": get_fundamental("Next report date"),
            "Report period": get_fundamental("Report period"),
            "EPS estimate": get_fundamental("EPS estimate"),
            "Revenue estimate": get_fundamental("Revenue estimate"),
            "Market capitalization": get_fundamental("Market capitalization"),
            "Dividend yield  (i)": get_fundamental("Dividend yield (i)"),
            "Price to earnings": get_fundamental("Price to earnings"),
            "Basic EPS  (TTM)": get_fundamental("Basic EPS (TTM)"),
            "Net income  (FY)": get_fundamental("Net income (FY)"),
            "Revenue  (FY)": get_fundamental("Revenue (FY)"),
            "Shares float": get_fundamental("Shares float"),
            "Beta  (1Y)": get_fundamental("Beta (1Y)"),
            "Employees  (FY)": get_fundamental("Employees (FY)"),
            "Change  (1Y)": get_fundamental("Change (1Y)")
        }
        output.append(row)
        print(f"✅ [{idx}/{len(symbols)}] {sym}")

    except TimeoutException:
        print(f"❌ [{idx}] Timeout loading {sym}")
        output.append({
            "Symbol": sym,
            "Next report date": "N/A",
            "Report period": "N/A",
            "EPS estimate": "N/A",
            "Revenue estimate": "N/A",
            "Market capitalization": "N/A",
            "Dividend yield  (i)": "N/A",
            "Price to earnings": "N/A",
            "Basic EPS  (TTM)": "N/A",
            "Net income  (FY)": "N/A",
            "Revenue  (FY)": "N/A",
            "Shares float": "N/A",
            "Beta  (1Y)": "N/A",
            "Employees  (FY)": "N/A",
            "Change  (1Y)": "N/A",
            "Error": "Page load timeout"
        })
    except Exception as e:
        print(f"❌ [{idx}] Error scraping {sym}: {e}")
        output.append({
            "Symbol": sym,
            "Next report date": "N/A",
            "Report period": "N/A",
            "EPS estimate": "N/A",
            "Revenue estimate": "N/A",
            "Market capitalization": "N/A",
            "Dividend yield  (i)": "N/A",
            "Price to earnings": "N/A",
            "Basic EPS  (TTM)": "N/A",
            "Net income  (FY)": "N/A",
            "Revenue  (FY)": "N/A",
            "Shares float": "N/A",
            "Beta  (1Y)": "N/A",
            "Employees  (FY)": "N/A",
            "Change  (1Y)": "N/A",
            "Error": str(e)
        })

# ─── Save to CSV ────────────────────────────────────────────────────────────────
fieldnames = [
    "Symbol", "Next report date", "Report period", "EPS estimate", "Revenue estimate",
    "Market capitalization", "Dividend yield  (i)", "Price to earnings", "Basic EPS  (TTM)",
    "Net income  (FY)", "Revenue  (FY)", "Shares float", "Beta  (1Y)", "Employees  (FY)",
    "Change  (1Y)", "Error"
]

with open("/Users/farazahmed/Documents/task3.csv", 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(output)

print("✅ Done! Data written to task3.csv")
driver.quit()


Total symbols read: 4576
❌ [1] Error scraping A: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=136.0.7103.114)
Stacktrace:
0   chromedriver                        0x0000000104c0c3e0 cxxbridge1$str$ptr + 2829900
1   chromedriver                        0x0000000104c046a8 cxxbridge1$str$ptr + 2797844
2   chromedriver                        0x0000000104741fbc cxxbridge1$string$len + 90140
3   chromedriver                        0x000000010471c798 chromedriver + 133016
4   chromedriver                        0x00000001047b1048 cxxbridge1$string$len + 544936
5   chromedriver                        0x00000001047c9b50 cxxbridge1$string$len + 646064
6   chromedriver                        0x000000010477d3f8 cxxbridge1$string$len + 332888
7   chromedriver                        0x0000000104bd0804 cxxbridge1$str$ptr + 2585200
8   chromedriver                        0x0000000104bd3ad4 cxxbridge1$str$ptr + 2598208
9   chromedriv

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=136.0.7103.114)
Stacktrace:
0   chromedriver                        0x0000000104c0c3e0 cxxbridge1$str$ptr + 2829900
1   chromedriver                        0x0000000104c046a8 cxxbridge1$str$ptr + 2797844
2   chromedriver                        0x0000000104741fbc cxxbridge1$string$len + 90140
3   chromedriver                        0x000000010471c798 chromedriver + 133016
4   chromedriver                        0x00000001047b1048 cxxbridge1$string$len + 544936
5   chromedriver                        0x00000001047c9b50 cxxbridge1$string$len + 646064
6   chromedriver                        0x000000010477d3f8 cxxbridge1$string$len + 332888
7   chromedriver                        0x0000000104bd0804 cxxbridge1$str$ptr + 2585200
8   chromedriver                        0x0000000104bd3ad4 cxxbridge1$str$ptr + 2598208
9   chromedriver                        